# importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
from math import sqrt

# Reading dataset and null checking

In [2]:
data = pd.read_csv("BankNote_Authentication.csv")
#normalized = pd.read_csv("BankNote_Authentication.csv")
print(data)
#print(f"null values: {data.isnull().sum()}")
data.info()

      variance  skewness  curtosis  entropy  class
0      3.62160   8.66610   -2.8073 -0.44699      0
1      4.54590   8.16740   -2.4586 -1.46210      0
2      3.86600  -2.63830    1.9242  0.10645      0
3      3.45660   9.52280   -4.0112 -3.59440      0
4      0.32924  -4.45520    4.5718 -0.98880      0
...        ...       ...       ...      ...    ...
1367   0.40614   1.34920   -1.4501 -0.55949      1
1368  -1.38870  -4.87730    6.4774  0.34179      1
1369  -3.75030 -13.45860   17.5932 -2.77710      1
1370  -3.56370  -8.38270   12.3930 -1.28230      1
1371  -2.54190  -0.65804    2.6842  1.19520      1

[1372 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   variance  1372 non-null   float64
 1   skewness  1372 non-null   float64
 2   curtosis  1372 non-null   float64
 3   entropy   1372 non-null   float64
 4   class     1372 non

# Spliting data
    divide dataset into 70% for training and 30% for testing.

In [3]:
X = data.drop(columns=['class'])
Y = data['class']
print(X)
#print(Y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=None)

      variance  skewness  curtosis  entropy
0      3.62160   8.66610   -2.8073 -0.44699
1      4.54590   8.16740   -2.4586 -1.46210
2      3.86600  -2.63830    1.9242  0.10645
3      3.45660   9.52280   -4.0112 -3.59440
4      0.32924  -4.45520    4.5718 -0.98880
...        ...       ...       ...      ...
1367   0.40614   1.34920   -1.4501 -0.55949
1368  -1.38870  -4.87730    6.4774  0.34179
1369  -3.75030 -13.45860   17.5932 -2.77710
1370  -3.56370  -8.38270   12.3930 -1.28230
1371  -2.54190  -0.65804    2.6842  1.19520

[1372 rows x 4 columns]


# Normalizating each feature separately:
    using the mean and std of the values of that feature column on the TRAINING data.

In [5]:
warnings.simplefilter('ignore')
print(x_test)
"""for i in range(x_train.shape[0]):
    for j in range(x_train.shape[1]):
        x_train.iloc[i,j] = (x_train.iloc[i,j] - x_train.iloc[: , j].mean() ) """

for i  in range(x_test.shape[0]):
    for j in range(x_test.shape[1]):
        x_test.iloc[i][j] = (x_test.iloc[i][j] - x_train.iloc[: , j].mean()) / x_train.iloc[: , j].std()

for i  in range(x_train.shape[0]):
    for j in range(x_train.shape[1]): 
        x_train.iloc[i][j] = (x_train.iloc[i][j] - x_train.iloc[: , j].mean()) / x_train.iloc[: , j].std()

    
    #x_train[i] = (x_train[i] - x_train[i].mean()) / x_train[i].std()
    #x_test[i] = (x_test[i] - x_train[i].mean()) / x_train[i].std()
    #print(x_test[i])

print("------------ After normalizing -------------- ")
print(x_test)

#first_element = y_train[0]
#print(first_element)

#print(x_train.info())
#print(x_test.info())


      variance  skewness  curtosis  entropy
1007  -0.17296   -1.1816   1.38180  0.73360
1176  -0.59587    2.4811  -2.86730 -0.89828
332    3.55940    1.3078   1.29100  1.65560
585    4.33980   -5.3036   3.88030 -0.70432
52     3.14520    5.8250  -0.51439 -1.49440
...        ...       ...       ...      ...
112    3.23510    9.6470  -3.20740 -2.59480
1214  -2.96620  -10.3257   8.78400 -2.11380
588   -0.27802    8.1881  -3.13380 -2.52760
984   -0.69745   -1.7672  -0.34474 -0.12372
1187  -4.02180   -8.3040  12.55500 -1.50990

[412 rows x 4 columns]
------------ After normalizing -------------- 
      variance  skewness  curtosis   entropy
1007 -0.201353 -0.529889 -0.001733  0.955527
1176 -0.351226  0.082355 -0.967885  0.180776
332   1.121348 -0.113770 -0.022379  1.393255
585   1.397912 -1.218908  0.566371  0.272860
52    0.974561  0.641310 -0.432885 -0.102238
...        ...       ...       ...       ...
112   1.006421  1.280182 -1.045216 -0.624664
1214 -1.191241 -2.058385  1.681365 -0.396

# KNN implementation:
     If there is a tie in the class predicted by the k -nearest neighbors, then among the classes that have the same number of votes, the tie should be broken in favor of the class that comes first in the Train file.

In [7]:
def euclidean_distance(x , data):
    return np.sqrt( np.sum( np.square( x - data ) ) )

class KnnClassifier():
    def __init__(self , knn=3 ,  parametric=euclidean_distance ):
        self.k=knn
        self.param = parametric
    
    def fit(self , x_train , y_train):
        self.x_train=x_train
        self.y_train = y_train
    
    def predict(self , x_test):
        hx=[]
        j=0
        for x1 in x_test.values:
            #print(x1)
            distances = np.zeros(self.x_train.shape[0])
            i=0
            for x2 in self.x_train.values:
                dist  = euclidean_distance(x1 , x2)
                distances[i] = dist
                i+=1
            sorted_inds = distances.argsort()
            y = [y for dis , y in sorted( zip(distances , self.y_train))]
            y = y[:self.k]


            n1 =0 
            n0=0
            for i in y:
                if i == 1:
                    n1+=1
                elif i == 0:
                    n0+=1
            if n1>n0:
                hx.append(1)
            elif n1==n0:
                hx.append(self.y_train.shape[0])
            else:
                hx.append(0)
         
        
        return np.array(hx)
    
    def accuracy(self , y , hx):
        return np.sum(y==hx) / len(y)


model = KnnClassifier(5)
model.fit(x_train , y_train)
hx = model.predict(x_test)

#for i in (x_train.values):
    #print(i)
#print(hx)

#print(y_test)

#print("Accuracy: " , model.accuracy(y_test , hx) * 100)





# Testing:
     Experiment with different values of k=1,2,3....9 and output The value of k used for the test set on the first line followed by summary info for the current k value: number of correctly classified test instances, and the total number of instances in the test set and accuracy.

In [9]:
for i in range(1 , 10 , 1):
    model = KnnClassifier(i)
    model.fit(x_train , y_train)
    hx = model.predict(x_test)

    print(f"k value: {i}")
    print(f"Total number of instances : {x_test.shape[0]}")
    print(f"Number of correctly classified instances : {np.sum(y_test==hx)} ")
    print(f"Accuracy : {model.accuracy(y_test , hx)}")
    print("----------------------------------------------------------")




k value: 1
Total number of instances : 412
Number of correctly classified instances : 408 
Accuracy : 0.9902912621359223
----------------------------------------------------------
k value: 2
Total number of instances : 412
Number of correctly classified instances : 408 
Accuracy : 0.9902912621359223
----------------------------------------------------------
k value: 3
Total number of instances : 412
Number of correctly classified instances : 408 
Accuracy : 0.9902912621359223
----------------------------------------------------------
k value: 4
Total number of instances : 412
Number of correctly classified instances : 406 
Accuracy : 0.9854368932038835
----------------------------------------------------------
k value: 5
Total number of instances : 412
Number of correctly classified instances : 406 
Accuracy : 0.9854368932038835
----------------------------------------------------------
k value: 6
Total number of instances : 412
Number of correctly classified instances : 404 
Accuracy 